In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
data = pd.read_csv("https://raw.githubusercontent.com/haram4th/ADsP/main/iris3.csv")
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [48]:
data['species'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [49]:
X = data.drop("species", axis=1)
y = data['species']

In [50]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()


y = le.fit_transform(y)


In [51]:
X = X.to_numpy().astype(np.float32)
y = y.astype(np.int32)  # 또는 np.float32도 가능

In [52]:
from sklearn.model_selection import train_test_split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [54]:
from sklearn.preprocessing import MinMaxScaler

In [55]:
mms = MinMaxScaler()
train_temp = mms.fit_transform(X_train)
test_temp = mms.transform(X_test)
mms_X_train = pd.DataFrame(train_temp, columns=X_train.columns, index=X_train.index)
mms_X_test = pd.DataFrame(test_temp, columns=X_test.columns, index=X_test.index)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [ ]:
mms_X_train

In [ ]:
mms_X_test

# Tensorflow로 다중분류 분석하기

Seqeuntial API

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input


In [ ]:
# 신경망 모델 정의 keras 3.0, tensorflow 2.19 기준
model = Sequential()
model.add(Input(shape=(mms_X_train.shape[1],))) # 입력층
model.add(Dense(16, activation='relu')) # 은닉층
model.add(Dense(8, activation='relu')) # 은닉층
model.add(Dense(4, activation='relu')) # 은닉층
model.add(Dense(3, activation='softmax')) # 출력층, 이진 분류이므로 sigmoid
model.summary()

* 다중분류의 경우 사용하는 loss
    *  categorical_crossentropy: 종속변수 클래스가 3개 이상인 경우이고 one_hot encoding 된 경우
    * sparse_categorical_crossentropy: 종속변수 클래스가 3개 이상인 경우이고 label encoding 된 경우

In [ ]:
# 모델의 파라미터 세팅
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer='adam', metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(mms_X_train, y_train, epochs=100, batch_size=16, 
                    validation_data=(mms_X_test, y_test))


pred = model.predict(mms_X_test)
result = pd.DateFrame(dict(y_test=y_test, pred=pred[:,0]))

plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

In [ ]:
print(np.unique(y_train, return_counts=True))
print(np.min(X_train), np.max(X_train))


In [ ]:
pred = model.predict(mms_X_test)
pred

In [ ]:
pd.DataFrame(pred[:,0])


Functional API

In [56]:
from keras.models import Model

In [ ]:
inputs = Input(shape=(mms_X_train.shape[1],))
x = Dense(16, activation='relu')(inputs)
x = Dense(8, activation='relu')(x)
x = Dense(4, activation='relu')(x)

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
history = model.fit(mms_X_train, y_train, epochs=100, batch_size=16,
                   validation_data=(mms_X_test, y_test))

pred = model.predict(mms_X_test)
pred = pd.DateFrame(dict(y_test=y_test, pred=pred[:,0]))
result = pd.concat([y_test, ])
plt.figure(figsize=(8, 5))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

Subclass API

In [57]:
class Iris(Model):
    def __init__(self):
        super(Iris, self).__init__()
        self.dense1 = Dense(16, activation='relu')
        self.dense2 = Dense(16, activation='relu')
        self.dense3 = Dense(16, activation='relu')
        self.classifier = Dense(3, activation='softmax')
        
    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        return self.classifier(x)

In [60]:
model2 = Iris()
model2.compile()

model.compile(loss="categorical_crossentropy", optimizer='adam', metrics=['accuracy'])
history = model2.fit(mms_X_train, y_train, epochs=100, batch_size=16,
                   validation_data=(mms_X_test, y_test))

pred = model2.predict(mms_X_test)
pred = pd.DateFrame(dict(y_test=y_test, pred=pred[:,0]))
result = pd.concat([y_test, pred], axis=1)
plt.figure(figsize=(8, 4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['train', 'val'])
plt.show()

Epoch 1/100


ValueError: No loss to compute. Provide a `loss` argument in `compile()`.